# Can We Predict the Weather Accurately Using Neural Networks?


<b><u>Abstract:</u></b><br>
Predicting the weather has been of importance to humans for thousands of years; it dictates how we make a multitude of decisions, varying from choosing what to wear in the morning to knowing when is a good time to plant crops. Currently, our weather predictions lie in the hands of meteorologists who use complicated models and algorithms based on concepts from physics, chemistry and mathematics in order to make their predictions. While the quality of these predictions has steadily improved over the years, the mathematically chaotic nature that weather patterns tend to follow make predictions extremely difficult, especially over long periods of time. Improving the accuracy of these predictions using machine learning models could be the next step towards advanced weather predictions. The work presented here shows that by using a model consisting of a combination of different nerual networks, reasonable predictions can be made. The quality of these predictions is perhaps hinting at the possibility of more accuracte models that could be developed in the near future. 

<table>
    <tr>
        <td>
            <figure>
                <img width=300 height=300 src="images/neural_network_brain.png">
            </figure>
        </td>
        <td>
            <figure>     
                <img src="images/weather_forcast.jpg">
           </figure>
        </td>
    </tr>
</table>



## Section 0: Imports, Functions, and Global Variables
As its name suggests, this section contains all of the imports, functions, and global variable declations that are used in order to complete this project. All of these items were included here to enhance readability of later sections and provide a single place where they can be referenced, if necessary. 

### 0.1: Imports
Below is a list of all imports that were used in order to complete this project, organized by category. 

In [ ]:
# Data manipulation imports
import pandas as pd
import numpy as np
np.random.seed(91993) 

In [ ]:
# Plotting imports 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Mathematical tools imports
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from scipy.integrate import odeint

In [ ]:
# Data Preprocessing Imports
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Machine Learning Algorithm Imports
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

In [ ]:
# Model selection (hyperparameter tuning) imports
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Neural Network Imports (Keras and tensorflow)
import keras
import tensorflow as tf
from keras.layers import Dense
from keras.models import Sequential
from keras.metrics import categorical_accuracy
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

### 0.2: Functions
Below is a list of all functions that were used in order to complete this project, listed in the order in which they are first used. 

In [ ]:
def make_classes(rainfall, snowfall):
    '''Creates target variable to predict by generating the following three classes from 
    rainfall and snowfall measurements:
    
    0 --> Clear weather, no rain or snow.
    1 --> Rainy weather.
    2 --> Snowy weather. 
    '''
    if snowfall == 0:
        if rainfall == 0:
            return 0
        else:
            return 1
    else:
        return 2
    
def make_classes_extra(rainfall, snowfall):
    '''Same as the 'make_classes' function, but adds an additional two classes based on the 
    severity of rainfall and snowdfall. Classes are as follows: 
    
    0 --> Clear weather, no rain or snow.
    1 --> Rain less than 0.3mm in the hour. Considered light rainfall.
    2 --> Rain more than 0.3mm in the hour. Considered moderate to heavy rainfall.
    3 --> Snow less than 0.2cm in the hour. Considered light snowfall.
    4 --> Snow more than 0.2cm in the hour. Considered moderate to heavy snowfall.
    '''
    if snowfall ==  0:
        if rainfall == 0:
            return 0
        else:
            if rainfall <= 0.3:
                return 1
            else:
                return 2
    else:
        if snowfall < 0.2:
            return 3
        else: 
            return 4
    
def get_data_ready(dataset):
    y_undersample = dataset[dataset['num_hours']==0]['class'].values.reshape(-1,1)
    y_undersample_cat = to_categorical(dataset[dataset['num_hours']==0]['class'])
    X_undersample = dataset[dataset['num_hours']==0][['temperature', 'humidity', 'mslp', \
                'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
                'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
                'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust']].values

    X_train, X_test, y_train, y_test = train_test_split(X_undersample, y_undersample_cat, test_size=0.3, \
                                                        random_state=rs)

### 0.3: Global Variables 
The cell below contains all global variables that were used in this project, in order to increase ease of reproducability. 

In [ ]:
rs = 42 # The random_state used by all relevant functions and methods in this project. 
seed_num = 91993 # The seed number for random calculations carried out by the numpy package. 
np.random.seed(seed_num) 

## Section 1: Introduction
This sections contains all introductory material that is referenced in later sections of this project. 
### 1.1: How are Weather Predictions Made?
Weather predictions are something that all of us rely on every day. In fact, the first thing most people do before heading out the door is check their favorite weather app to determine what they should be wearing that day. While we are surrounding by weather predictions every day, it turns out that the models used to calculate these predictions are extemeley complex. While there are a multitude of different methods for doing this, they almost always rely on some implementation of the Navier-Stokes equations. These equations are the foundation of fluid dynamics, similar to how the Schrodinger equation is the foundation of quantum mechanics. While these equaitons are a great achievement of physics, the fact that they are intricate differential equations makes them difficult to solve. One version of the equations can be seen below, provided by NASA: 

<img width=700 height=700 src="images/navier_stokes_equations.gif"/>

While the equations do provide information about the temperature, pressure, momentum, and density of a moving fluid, there is a lot of computational work that needs to be done in order to solve them. <br>

In addition to the complex nature of the Navier Stokes equations, weather systems are also considered to be chaotic systems. Mathematically speaking, chaotic systems exhibit the following traits:

<ol>
    <li> The system is deterministic.</li>
    <li> The system is aperiodic.</li>
    <li> The system is extremely sensitive to intitial conditions.</li>
</ol>
        
The first trait might not make sense at first, as it might seem that a chaotic system is one that shouldn't be able to be perfectly described (deterministic). However, this is a common misconception: chaotic systems are entirely predictable, even for long periods of time, if you have the correct inputs. The real difficulty in modeling chaotic systems results from the third trait: chaotic systems are extremeley sensitive to initial conditions, meaning that two slightly different sets of inputs can result is drastically different predictions. This means that no matter how close a meterologist is to getting spot on atmospheric data, even the smallest discrepensy from the actual model can cause predictions to be way off in the near future. This is sometimes more colloquially referred to as the "butterfly effect." The following aside gives an example of a more simplistic chaotic system. 

### Aside: A Simple Chaotic System - The Lorenz System
One of the very first systems that is studied in any chaos theory class in known as the Lorenz system, first studied in detail by Edward Lorenz. This system relates to the project presented here, as he studied it in an attempt to better understand atmospheric convection. The system consists of three ordinary differential equations, which are shown below:
$$
\begin{align}
\frac{dx}{dt} &= \sigma(y-x) \\
\frac{dy}{dt} &= x(\rho-z)-x \\
\frac{dz}{dt} &= xy - \beta z
\end{align}
$$
In the equations above, $x$, $y$, and $z$ are cartesian coordinates, while $\sigma$, $\rho$, and $\beta$ are system parameters. This system is translated into python in the cell below:

In [ ]:
def dx_dt(x, y):
    return sigma * (y - x)
def dy_dt(x, z):
    return x * (rho - z) - x
def dz_dt(x, y, z):
    return x * y - beta * z
def lorenz_system(coordinates, t):
    x, y, z = coordinates
    return dx_dt(x, y), dy_dt(x, z), dz_dt(x, y, z) 

It turns out that for most values of $\sigma$, $\rho$, and $\beta$, the Lorenz system is not chaotic at all. More specifically, if $\rho$ is small, the system almost always converges to one or two fixed points. However, once $\rho$ is larger than $\approx$ 24.74, chaotic behavior starts to occur. Feel free to change the intial conditions ($\sigma$, $\rho$, and $\beta$) in the cell below in order to see how the behavior of the lorenz system changes. Some suggestions are given below:
<ul>
    <li>$\rho$=10, $\sigma$=10, $\beta=\frac{8}{3}$: To see how the lorenz system behaves when it is not chaotic.</li>
    <li>$\rho$=28, $\sigma$=10, $\beta=\frac{8}{3}$: To see how the lorenz system behaves when it is not chaotic.</li>
    <li>$\rho$=28.1, $\sigma$=10, $\beta=\frac{8}{3}$: To see how a small change to a chaotic system produces an entirely different result.</li>
</ul>

In [ ]:
# Try changing these three variables
rho = 10.0
sigma = 10.0
beta = 8.0 / 3.0

# Do not change these...
t_i = 0
t_f = 20
t_step = 0.01
t = np.arange(t_i, t_f, t_step)
initial_xyz = [1.0, 1.0, 1.0]

def plot_lorenz(initial_xyz, t):
    positions = odeint(lorenz_system, initial_xyz, t)
    plt.axes(projection='3d')
    plt.title('Lorenz System')
    plt.plot(positions[:,0], positions[:,1], positions[:,2])
    
plt.figure(figsize=(12,10))
plot_lorenz(initial_xyz, t)

In case its too hard to make out the difference, the cell below shows the Lorenz system when $\rho$=28.0, $\sigma$=10.0, $\beta=\frac{8}{3}$, and when $\rho$=28.1, $\sigma$=10.0, $\beta=\frac{8}{3}$.

<table>
    <tr>
        <td>
            <figure>
                <img src="images/lorenz1.png">
                <figcaption><center>Initial Conditions: $\rho$=28.0, $\sigma$=10.0, and $\beta=\frac{8}{3}$.</center></figcaption>   
            </figure>
        </td>
        <td>
            <figure>     
                <img src="images/lorenz2.png">
                <figcaption><center>Initial Conditions: $\rho$=28.1, $\sigma$=10.0, and $\beta=\frac{8}{3}$.</center></figcaption>
           </figure>
        </td>
    </tr>
</table>


While the general shape of the graph is the same, further inspection reveals many differences by simply shifting $\rho$ by only 0.1 Another way of looking at this is to examine the final points of the lorenz system using each set of initial conditions. This is done in the cell below: 

In [ ]:
rho = 28.0
sigma = 10.0
beta = 8.0 / 3.0
positions1 = odeint(lorenz_system, initial_xyz, t)
rho = 28.1
sigma = 10.0
beta = 8.0 / 3.0
positions2 = odeint(lorenz_system, initial_xyz, t)
print('Last Positions of Lorenz system with rho = 28.0: ', positions1[-1])
print('Last Positions of Lorenz system with rho = 28.1: ', positions2[-1])
print(mean_squared_error(positions1, positions2))

We see that the final points from each system are completely different despite their initial conditions being nearly identical. Lastly, we can look at the mean squared error between the points from the two different Lorenz systems that were calculated:

In [ ]:
print('Mean Squared Error between the two lorenz systems:', mean_squared_error(positions1, positions2))

As is clear from the printout above, despite their shape looking similar, the points from the two different systems have an exorbitantly high mean squared error.

The point in showing this is to demonstrate that this sensitivity to inital conditions for chaotic systems can make correct predictions extremely difficult. The systems that dictate our weather patterns are also much more complex, further adding to the issue. In summary, this means is that if the atmospheric data used to make predictions is only slightly different from the actual value, the predictions might not be close to the weather that's actually on its way. This sensitivity also gets worse over time, which accounts for the fact that weather predictions made far in advance have a high probability of being incorrect. 

### 1.2: Neural Networks, a Possible Solution?

The previous subsection described how the current methods that are used in making weather predictions have some pretty serious flaws. The work performed here is meant to test whether or not predictive algorithms (specifically, neural networks) can be used to make predictions that are as good, or possibly even better than the physical models that are currently in use. <br><br>

Nueral networks seemed like a reliable choice to attempt to solve this problem, as they are capable of being able to determine complicated linear and non-linear relationships between a large number of input variables. Because the current method of weather forecasting is also complex, there already exists a large amount of high quality atmospheric data being used to make the predictions as accurate as possible. The work done in this project simply applies that same data to different machine learning models, with the most accurate possible neural network as the desired product. In general, these models performed reasonably well. 

### 1.3: The Data
The data used in this project comes from <a href="https://www.meteoblue.com/en/weather/week">meteoblue</a>, a meteorological service created at the University of Basel. They provide current, high quality meteorological data from places all over the world, but also give access to free historical weather data for Basel, Switzerland dating back to 1984. This was the data used for this project, and can be accessed <a href="https://www.meteoblue.com/en/products/historyplus/download/basel_switzerland_2661604">here.</a> A list of all the variables that are included in the dataset, along with their units and a brief description, is listed in the table below:

<table style="width:80%">
  <tr>
    <th>Measurement</th>
    <th>Units</th> 
    <th>Description</th>
  </tr>
  <tr>
    <td>Temperature</td>
    <td>$^{\circ}$F</td> 
    <td>The temperature recorded from 2 meters above the ground.</td>
  </tr>
  <tr>
    <td>Relative humidity</td>
    <td>%</td> 
    <td>the amount of water vapor present in air expressed as a percentage of the amount needed for saturation at the same temperature</td>
  </tr>
  <tr>
    <td>Pressure</td>
    <td>hPa</td> 
    <td>The mean sea level pressure (MSLP).</td>
  </tr>
  <tr>
    <td>Precipitation Amount</td>
    <td>mm</td> 
    <td>The total amount of precipitation that fell in hour measured according to a high resolution model.</td>
  </tr>
  <tr>
    <td>Precipitation amount (low resolution)</td>
    <td>mm</td> 
    <td>The total amount of precipitation that fell in hour measured according to a low resolution model.</td>
  </tr>
  <tr>
    <td>Snowfall amount (high resolution)</td>
    <td>cm</td> 
    <td>The total amount of snowdall that fell in the hour measured according to a high resolution model.</td>
  </tr>
  <tr>
    <td>Snowfall amount (low resolution)</td>
    <td>cm</td> 
    <td>The total amount of precipitation that fell in the hour measured according to a low resolution model.</td>
  </tr>
  <tr>
    <td>Total cloud cover</td>
    <td>%</td> 
    <td>The fraction of the sky covered by a cloud of any type at any height in the sky.</td>
  </tr>
  <tr>
    <td>High cloud cover</td>
    <td>%</td> 
    <td>The fraction of the sky covered by clouds at high elevation (5-13 km).</td>
  </tr>
  <tr>
    <td>Mid cloud cover</td>
    <td>%</td> 
    <td>The fraction of the sky covered by clouds at medium elevation (2-7 km).</td>
  </tr>
  <tr>
    <td>Low cloud cover</td>
    <td>%</td> 
    <td>The fraction of the sky covered by clouds at low elevation ($<$2 km).</td>
  </tr>
  <tr>
    <td>Sunshine duration (minutes)</td>
    <td>Minutes</td> 
    <td>The number of minutes in each hour when there was no cloud cover.</td>
  </tr>
  <tr>
    <td>Solar radiation</td>
    <td>W/m$^2$</td> 
    <td>The power emitted by the sun for every sqaure meter. Also referred to as 'irradiance'.</td>
  </tr>
  <tr>
    <td>Wind speed (10m)</td>
    <td>mph</td> 
    <td>The wind speed measured from 10m above the ground.</td>
  </tr>
  <tr>
    <td>Wind direction (10m)</td>
    <td>$^\circ$ (0$^{\circ}$ is North)</td> 
    <td>The wind direction measured from 10m above the ground.</td>
  </tr>
  <tr>
    <td>Wind speed (80m)</td>
    <td>mph</td> 
    <td>The wind speed measured from 80m above the ground.</td>
  </tr>
  <tr>
    <td>Wind direction (80m)</td>
    <td>$^{\circ}$ (0$^{\circ}$ is North)</td> 
    <td>The wind direction measured from 10m above the ground.</td>
  </tr>
  <tr>
    <td>Wind speed (900hPa)</td>
    <td>mph</td> 
    <td>The wind speed measured at an air pressure of 900hPa.</td>
  </tr>
  <tr>
    <td>Wind direction (900hPa)</td>
    <td>$^{\circ}$ (0$^{\circ}$ is North)</td> 
    <td>The wind direction measured at an air pressure of 900hPa.</td>
  </tr>
  <tr>
    <td>Wind gusts (10m)</td>
    <td>mph</td> 
    <td>Highest wind speed that was measured in an hour from 10m above the ground. </td>
  </tr>
</table>

## Section 2: Data Cleaning
This section includes all data cleaning steps that were taken in order to prepare the dataset for further analysis. Luckily, not much extensive cleaning was required, given the high quality of data provided by <a href="https://www.meteoblue.com/en/weather/week">meteoblue</a>. The raw data can be accessed from the following <a href="basel_weather_7-1-09_to_7-1-19.csv">file</a>, and contains ten years of hourly atmospheric data from Basel, Switzerland dating from 7/1/09 to 7/1/19.

In [ ]:
# Read in dataset and convert into a pandas dataframe. 
df = pd.read_csv('basel_weather_7-1-09_to_7-1-19.csv', sep=';', header=10)
df.head()

In [ ]:
# Print out dataframe information. 
df.info()

As can be seen in the prinout above, all variables that are listed in Section 1.3 are present, and none of the columns contain any null values. 

In [ ]:
df['Minute'].value_counts()

Since all measurements are taken at the top of every hour, the minute column provides no additional information (it is always zero) and can be dropped. This is clear from the `.value_counts()` printout above.

In [ ]:
# Drop minute column.
df = df.drop(columns=['Minute'])
df.head()

In [ ]:
# Rename the columns so they are clearer and more concise. 
column_names = ['year', 'month', 'day', 'hour', 'temperature', 'humidity', 'mslp', 'rainfall_hr', 'rainfall_lr', \
                'snowfall_hr', 'snowfall_lr', 'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
                'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
                'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust']
df.columns = column_names
df.head()

Note that both the rainfall and snowfall measurements come in both high resoluton and low resolution options. In order to choose between the two, a direct inquiry to meteoblue was made. According to one of their representatives:

<b>"For precipitation and snowfall amount, we (usually) offer two different datasets:
<ol>
    <li>The low resolution data, available since 1985, comes from our global NEMS30 (30km resolution) model, and is available for every point on earth.</li>
    <li>The high resolution data depends on the location you are collecting the data for. For all regions where we run a local domain of our model, data with a higher resolution is available. On the following <a href="https://content.meteoblue.com/en/specifications/data-sources/weather-simulation-data/meteoblue-models"/>page</a>, you can see which local domain is available for which region and when that it was implemented.</li>
</ol>
If there is no high resolution domain available, both options will provide the data from NEMS30.
We offer these two datasets for precipitation because, opposite to other variables such as temperature, it is not possible to merge two different datasets so that there is no inconsistency remaining."</b>

The following chart can be obtained by going to the page referenced in the quote above:
<img width=700 height=700 src="images/meteoblue_data_types.png"/>
Based on this chart, we see that in 2008 meteoblue starting using the high resolution NMM12 model in order to measure rainfall and snowfall amounts in Europe (and thus, Switzerland). Since all the data used in this project is from 2009 or later, using the high resolution data seemed like the better choice. The cell below removes the low resolution data so that only the high resolution data remains.



In [ ]:
# Use only the high resolution rainfall and snowfall measurements.
df = df.drop(columns = ['rainfall_lr', 'snowfall_lr'])
df.head()

The times that were provided in the dataset were given in Greenwich mean time (GMT), which is two hours behind the time in Basel, Switzerland. The following cell accounts for this difference by shifting the time two hours forward. 

In [ ]:
# Account for time difference between GMT and time in Basel.
df['hour'] = df['hour'].map(lambda x: (x + 2) % 24)
for i, row in df.iterrows():
      if row.hour == 0 or row.hour == 1:
        df.at[i,'day'] += 1
df.head()

The <b>rainfall_hr</b> and <b>snowfall_hr</b> columns give the respective snowfall that occured in the hour after other measurements like temperature and humidity were taken. In the cell below, the values in these columns are used in conjunction with the `make_classes()` function in order to create a new column, called <b>class_0hr</b>, that will act as the target variable in future predictive modeling. The three classes are listed below (which can also be found in the DOCSTRING of the `make_classes()` function):
<table style="width:30%">
  <tr>
    <th>Class</th> 
    <th>Description</th>
  </tr>
  <tr>
    <td>0</td>
    <td>Clear Weather - no rain or snow.</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Rainy weather - It rained (but did not snow) any amount in the hour.</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Snowy weather - It snowed any amount in the hour.</td>
  </tr>
</table>


In [ ]:
# Create classes for the rows.
df['class_0hr'] = df.apply(lambda x: make_classes(x.rainfall_hr, x.snowfall_hr), axis = 1)
df.head()

The dataframe seen above now incldues classes for each of its rows. However, in that current state any model that uses it would only be able to make weather predictions within 1 hour of the other quantities being measured. The cell below adds an additional 12 columns that allow any model to make predictions up to 12 hours in the future. In other words, it changes the dataframe so that each row includes if it was clear, rainy, snowy for 0-12 hours in the future.

In [ ]:
for i in range(1, 13):
    col_name = 'class_' + str(i) + 'hr'
    shiftnum = i * -1
    df[col_name] = df['class_0hr'].shift(shiftnum)
    df = df.dropna()
    df[col_name] = df[col_name].astype(int)


df.tail()

In [ ]:
df.info()

From the `.info()` printout above, we now see that each row contains a class that gives the type of weather for 0, 1, 2, all the way up to 12 hours into the future. More specifically, <b>class_0hr</b> says whether it was clear, rainy or snowy between 0-1 hours of the measurements being taken, <b>class_1hr</b> says whther it was clear rainy or snowy between 1-2 hours of the measurements being taken, and so on. 

While there are no issues with bad or missing data, the cell below exposes another problem particular to this dataset:

In [ ]:
plt.figure(figsize=(10,8))
df['class_0hr'].value_counts().plot(kind='bar', title='Count - Class');
plt.title('Class Count')
plt.xlabel('Class');
plt.ylabel('Count');

count_class_0, count_class_1, count_class_2 = df['class_0hr'].value_counts()

plt.ylim([0, max([count_class_0, count_class_1, count_class_2])*1.15])
plt.text(-0.10, count_class_0-1000, count_class_0, fontweight='bold')
plt.text(-0.10, count_class_0+1000, 'None', color='orange', fontweight='bold')
plt.text(0.90, count_class_1-1000, count_class_1, fontweight='bold')
plt.text(0.90, count_class_1+1000, 'Rain', color='orange', fontweight='bold')
plt.text(1.90, count_class_2-500, count_class_2, fontweight='bold')
plt.text(1.90, count_class_2+1000, 'Snow', color='orange', fontweight='bold');

Using the <b>class_0hr</b> column as an exmaple, it is clear according to the bar graph above that the vast majority of all rows represent hours in which the weather was clear. This can wreck havok on a model's ability to make accurate predicitons, as having discrepencies in the frequency of different classes can cause the model to be more or less likely to choose a certain class. 

In [ ]:
df_class_0 = df[df['class_0hr'] == 0]
df_class_1 = df[df['class_0hr'] == 1]
df_class_2 = df[df['class_0hr'] == 2]

count_class_0, count_class_1, count_class_2 = df['class_0hr'].value_counts()

In [ ]:
df_under_0 = df_class_0.sample(count_class_2)
df_under_1 = df_class_1.sample(count_class_2)
df_undersample = pd.concat([df_under_0, df_under_1, df_class_2], axis=0).reset_index(drop=True)

plt.figure(figsize=(10,8))
df_undersample['class_0hr'].value_counts().plot(kind='bar');
plt.title('Class Count Using Undersampling')
plt.xlabel('Class');
plt.ylabel('Count');

count0, count1, count2 = df_undersample['class_0hr'].value_counts()

plt.ylim([0, max([count0, count1, count2])*1.15])
plt.text(1.95, count0-25, count0, fontweight='bold')
plt.text(1.925, count0+25, 'None', color='orange', fontweight='bold')
plt.text(0.95, count1-25, count1, fontweight='bold')
plt.text(0.925, count1+25, 'Rain', color='orange', fontweight='bold')
plt.text(-0.05, count2-25, count2, fontweight='bold')
plt.text(-0.075, count2+25, 'Snow', color='orange', fontweight='bold');

In [ ]:
plt.figure(figsize=(15,50))
for i in range(1, 13):
    plt.subplot(6, 2, i)
    col_name = 'class_' + str(i) + 'hr'
    df_undersample[col_name].value_counts().plot(kind='bar')
    plt.title('Class Count - %d Hour Ahead' %i)
    plt.grid()
    plt.ylim([0,3000])
    plt.xlabel('Class')
    plt.ylabel('Count')

In [ ]:
df_list = []
for i in range(0, 13):
    col_name = 'class_' + str(i) + 'hr'
    cols = ['year', 'month', 'day', 'hour', 'temperature', 'humidity', 'mslp', 'rainfall_hr', \
            'snowfall_hr', 'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
            'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
            'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust', col_name]
    tmp_df = df[cols]
    tmp_df = tmp_df.rename(index=str, columns={col_name: 'class'})
    df_class_0 = tmp_df[tmp_df['class'] == 0]
    df_class_1 = tmp_df[tmp_df['class'] == 1]
    df_class_2 = tmp_df[tmp_df['class'] == 2]
    
    count_class_0, count_class_1, count_class_2 = tmp_df['class'].value_counts()
    df_under_0 = df_class_0.sample(count_class_2)
    df_under_1 = df_class_1.sample(count_class_2)
    class_df = pd.concat([df_under_0, df_under_1, df_class_2], axis=0).reset_index(drop=True)
    class_df['num_hours'] = i
    df_list.append(class_df)

df_us = pd.concat(df_list).reset_index(drop=True)
df_us.head()

In [ ]:
df_us['class'].value_counts()

In [ ]:
df_us.info()

# Section 3: Exploratory Data Analysis

In [ ]:
cols_for_hist = ['temperature', 'humidity', 'mslp', 'rainfall_hr', \
                'snowfall_hr', 'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
                'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
                'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust']
plt.figure(figsize=(15,60))
plotnum = 1
for column in cols_for_hist:
    plt.subplot(len(cols_for_hist)/2, 2, plotnum)
    plotnum += 1
    plt.hist(x=df_us[df_us['num_hours']==0][column], bins='auto')
    plt.grid()
    plt.title('Histogram - {}'.format(column), fontweight='bold')
    plt.ylabel('Count')
    plt.xlabel(column)

In [ ]:
x_scatter_cols = ['temperature', 'humidity', 'mslp', \
                 'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
                 'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
                 'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust']
y_scatter_cols = ['rainfall_hr', 'snowfall_hr']
plt.figure(figsize=(15,120))
plotnum = 1
for x_col in x_scatter_cols:
    for y_col in y_scatter_cols:
        plt.subplot(len(x_scatter_cols), len(y_scatter_cols), plotnum)
        plotnum +=1
        r = pearsonr(x=df_us[df_us['num_hours']==0][x_col], y=df_us[df_us['num_hours']==0][y_col])[0]
        plt.scatter(x=df_us[df_us['num_hours']==0][x_col], y=df_us[df_us['num_hours']==0][y_col], \
                   label='Pearson Correlation Coefficient (r) = {}'.format(r))
        plt.legend()
        plt.ylim([0, max(df_us[df_us['num_hours']==0][y_col])*1.15])
        plt.grid()
        plt.title('Scatterplot - %s vs. %s' %(x_col, y_col), fontweight='bold')
        plt.ylabel(y_col)
        plt.xlabel(x_col)

In [ ]:
y_undersample = df_us[df_us['num_hours']==0]['class'].values
y_undersample_cat = to_categorical(df_us[df_us['num_hours']==0]['class'])
X_undersample = df_us[df_us['num_hours']==0][['temperature', 'humidity', 'mslp', \
                'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
                'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
                'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust']].values

n_inputs = X_undersample.shape[1]
n_outputs = len(df_us[df_us['num_hours']==0]['class'].unique())

X_train, X_test, y_train, y_test = train_test_split(X_undersample, y_undersample, test_size=0.3, \
                                                            random_state=rs)
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_undersample, y_undersample_cat, \
                                                                            test_size=0.3, random_state=rs)

X_undersample.shape

In [ ]:
plt.figure(figsize = (10, 8))
max_n = 15
neighbors = np.arange(1, max_n+1)
train_scores = np.empty(len(neighbors))
test_scores = np.empty(len(neighbors))
for i, k in enumerate(neighbors):
    knn_tmp = KNeighborsClassifier(n_neighbors = k)
    knn_tmp.fit(X_train, y_train)
    train_scores[i] = knn_tmp.score(X_train, y_train)
    test_scores[i] = knn_tmp.score(X_test, y_test)

plt.plot(neighbors, train_scores, label = 'Training Set Performance')
plt.plot(neighbors, test_scores, label = 'Test Set Performance')
plt.legend(loc='upper right')
plt.xlabel('Neighbors (n)')
plt.ylabel('Accuracy Score')
plt.title('K-Nearest Neighbor Performace (No Preprocessing)');

In [ ]:
best_test_score = max(test_scores)
best_k = neighbors[np.where(test_scores == best_test_score)][0]
print('K-Nearest Neighbor Model Results (No Preprocessing):\n')
print('Best Test Data Performance: %3.2f' %best_test_score)
print('Optimal Number of Neighbors: %d' % best_k)


In [ ]:
plt.figure(figsize = (10, 8))
r = pearsonr(df_undersample.humidity, df_undersample.temperature)[0]
plt.scatter(x=df_undersample.humidity, y=df_undersample.temperature, label =\
         'Pearson Correlation Coefficient: %2.2f' %r)
plt.title('Scatterplot - Temperature vs. Humidity')
pearsonr(df_undersample.humidity, df_undersample.temperature)[0]
plt.xlabel('Humidty %')
plt.legend()
plt.grid()
plt.ylim([0, max(df_undersample.temperature)*1.15])
plt.ylabel('Temperature (F)');
#pearsonr(df_undersample.humidity, df_undersample.temperature)[0]

In [ ]:
scaler = StandardScaler()
pca = PCA()
pipeline = make_pipeline(scaler, pca)
pipeline.fit(X_undersample)

plt.figure(figsize=(15,8))
features = range(pca.n_components_)

plt.subplot(1, 2, 1)
plt.bar(features, pca.explained_variance_)
plt.title('Variance of PCA Features', fontweight='bold')
plt.xlabel('PCA Feature')
plt.ylabel('Variance')
plt.grid()
plt.xticks(features)


plt.subplot(1, 2, 2)
plt.plot(features, np.cumsum(pca.explained_variance_ratio_))
plt.title('Percentage of Variance Explained By Different Numbers of PCA Features', fontweight='bold')
plt.xlabel('PCA Feature')
plt.axhline(y=0.95, color = 'red', linestyle='--', label='95% Boundary')
plt.legend()
plt.ylabel('Fraction of Explained Variance')
plt.grid()
plt.xticks(features);

In [ ]:
total = 0
best_n = 0
for percent in pca.explained_variance_ratio_:
    total += percent
    best_n += 1
    if total >= 0.95:
        break

print('%d PCA Features explain %2.2f %% of the variance.' %(best_n, total*100))

In [ ]:
scaler = StandardScaler()
pca = PCA(n_components=best_n)
pipeline = make_pipeline(scaler, pca)
pca_features = pipeline.fit_transform(X_undersample)
target = df_us[df_us['num_hours']==0]['class'].values

pca_X_train, pca_X_test, y_train, y_test = train_test_split(pca_features, target, test_size=0.3, \
                                                            random_state=rs)

In [ ]:
plt.figure(figsize = (10, 8))
max_n = 15
neighbors = np.arange(1, max_n+1)
train_scores = np.empty(len(neighbors))
test_scores = np.empty(len(neighbors))
for i, k in enumerate(neighbors):
    knn_tmp = KNeighborsClassifier(n_neighbors = k)
    knn_tmp.fit(pca_X_train, y_train)
    train_scores[i] = knn_tmp.score(pca_X_train, y_train)
    test_scores[i] = knn_tmp.score(pca_X_test, y_test)

plt.plot(neighbors, train_scores, label = 'Training Set Performance')
plt.plot(neighbors, test_scores, label = 'Test Set Performance')
plt.legend(loc='upper right')
plt.xlabel('Neighbors (n)')
plt.ylabel('Accuracy Score')
plt.title('K-Nearest Neighbor Performance (with Preprocessing)');

In [ ]:
best_test_score = max(test_scores)
best_k = neighbors[np.where(test_scores == best_test_score)]
print('K-Nearest Neighbor Model Results (with Preprocessing):\n')
print('Best Test Data Performance: %3.2f' %best_test_score)
print('Optimal Number of Neighbors: %d' % best_k)

In [ ]:
tmp = []
for i in range(0, 13):
    subset = df_us[df_us['num_hours']==i]
    y = subset['class'].values
    X = subset.drop(columns=['num_hours', 'class', 'rainfall_hr', 'snowfall_hr', \
                             'year', 'month', 'day', 'hour']).values
    
    scaler = StandardScaler()
    pca = PCA()
    pipeline = make_pipeline(scaler, pca)
    pipeline.fit(X)
    best_n = 0
    total = 0
    for percent in pca.explained_variance_ratio_:
        total += percent
        best_n += 1
        if total >= 0.95:
            break
    scaler = StandardScaler()
    pca = PCA(n_components=best_n)
    pipeline = make_pipeline(scaler, pca)
    X_transform = pipeline.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_transform, y, test_size=0.3, random_state=rs)
    
    max_n = 15
    neighbors = np.arange(1, max_n+1)
    train_scores = np.empty(len(neighbors))
    test_scores = np.empty(len(neighbors))
    for j, k in enumerate(neighbors):
        knn_tmp = KNeighborsClassifier(n_neighbors = k)
        knn_tmp.fit(X_train, y_train)
        train_scores[j] = knn_tmp.score(X_train, y_train)
        test_scores[j] = knn_tmp.score(X_test, y_test)
    best_test_score = max(test_scores)
    best_train_score = max(train_scores)
   
    tmp.append([i, best_train_score, best_test_score])

tmp = np.asarray(tmp)
KNN_performances_df = pd.DataFrame({'num_hours': tmp[:,0], 'best_train_performance': tmp[:,1],\
                                        'best_test_performance': tmp[:,2]})

In [ ]:
plt.figure(figsize = (10, 8))
plt.plot(KNN_performances_df.num_hours, KNN_performances_df.best_test_performance)
plt.ylim([0.5, 1])
plt.grid()
plt.xlabel('Number of Hours to Prediction')
plt.ylabel('Best KNN Test Set Score')

plt.title('KNN Performances at Different Prediction Time Intervals');

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_undersample, y_undersample, test_size=0.3, \
                                                    random_state=rs)


rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
print('Out of the box score for Random Forest Classifier: {}'.format(rfc.score(X_test, y_test)))

In [ ]:
param_grid = {
    'max_depth': [2, 10, 100],
    'n_estimators': [50, 100, 200],
    'min_samples_leaf': np.arange(2, 6),
    'min_samples_split': np.arange(2, 6)
}


gscv = GridSearchCV(rfc, param_grid, cv=3)
gscv.fit(X_train, y_train)

print(gscv.best_params_)

In [ ]:
rfc = RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=5, n_estimators=100)
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

In [ ]:
feature_importances = pd.DataFrame(rfc.feature_importances_, \
                                   index = df_us[df_us['num_hours']==0].drop(columns=['class', 'num_hours', 'year', 'month', 'day', 'hour', 'rainfall_hr', 'snowfall_hr']).columns, \
                                   columns=['importance']).sort_values('importance', ascending=False)

In [ ]:
plt.figure(figsize=(10,8))
plt.barh(feature_importances.index, feature_importances.importance.values)
plt.xlabel('Feature Importance')
plt.ylabel('Feature Name')
plt.title('Feature Importance Using Random Forest Classifer');

In [ ]:
svc = SVC()
svc.fit(pca_X_train, y_train)
predictions = svc.predict(pca_X_test)
svc.score(pca_X_test, y_test)


In [ ]:
param_dist = {'C': np.logspace(-6, 6, 13),
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
              'gamma': np.logspace(-6, 6, 13),
              'tol': np.logspace(-6, 6, 13)}

#svc = SVC()
#best_svc = RandomizedSearchCV(svc, param_dist, cv=3)
#best_svc.fit(pca_X_train, y_train)
#print("Tuned SVC Parameters: {}".format(best_svc.best_params_))
#print("Best score is {}".format(best_svc.best_score_))

In [ ]:
#best_svc.score(pca_X_test, y_test)

# Section 4: In-Depth Analysis

In [ ]:
y_undersample = df_us[df_us['num_hours']==0]['class'].values.reshape(-1,1)
y_undersample_cat = to_categorical(df_us[df_us['num_hours']==0]['class'])
X_undersample = df_us[df_us['num_hours']==0][['temperature', 'humidity', 'mslp', \
                'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
                'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
                'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust']].values

X_train, X_test, y_train, y_test = train_test_split(X_undersample, y_undersample_cat, test_size=0.3, \
                                                    random_state=rs)

## Using Tensorflow

In [ ]:
X_scaler = MinMaxScaler(feature_range=(0,1))

X_scaled_training = X_scaler.fit_transform(X_train)
X_scaled_testing = X_scaler.transform(X_test)

In [ ]:
learning_rate = 0.001
training_epochs = 100
display_step = 5

In [ ]:
num_inputs = X_train.shape[1]
num_outputs = 3

In [ ]:
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

In [ ]:
tf.reset_default_graph()

In [ ]:
with tf.variable_scope('inupt'):
    X = tf.placeholder(tf.float32, shape=(None, num_inputs))

with tf.variable_scope('layer1'):
    weights = tf.get_variable(name='weights1', shape=[num_inputs, layer_1_nodes], \
                              initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_out = tf.nn.relu(tf.matmul(X, weights) + biases)

with tf.variable_scope('layer2'):
    weights = tf.get_variable(name='weights2', shape=[layer_1_nodes, layer_2_nodes], \
                             initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_out = tf.nn.relu(tf.matmul(layer_1_out, weights) + biases)

with tf.variable_scope('layer3'):
    weights = tf.get_variable(name='weights3', shape=[layer_2_nodes, layer_3_nodes], \
                             initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_out = tf.nn.relu(tf.matmul(layer_2_out, weights) + biases)

with tf.variable_scope('output'):
    weights = tf.get_variable(name='weights4', shape=[layer_3_nodes, num_outputs], \
                             initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape=[num_outputs], initializer=tf.zeros_initializer())
    prediction = tf.nn.softmax(tf.matmul(layer_3_out, weights) + biases)

with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 3))
    cost = tf.losses.softmax_cross_entropy(Y, prediction)

# define optimzier
with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()

In [ ]:
with tf.Session() as session:

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())

    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    training_writer = tf.summary.FileWriter('./logs/training', session.graph)
    testing_writer = tf.summary.FileWriter('./logs/testing', session.graph)

    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):

        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X: X_scaled_training, Y: y_train})

         # Every few training steps, log our progress
        if epoch % display_step == 0:
            # Get the current accuracy scores by running the "cost" operation on the training and test data sets
            training_cost, training_summary = session.run([cost, summary], feed_dict={X: X_scaled_training, Y:y_train})
            testing_cost, testing_summary = session.run([cost, summary], feed_dict={X: X_scaled_testing, Y:y_test})

            # Write the current training status to the log files (Which we can view with TensorBoard)
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)

            # Print the current training status to the screen
            print("Epoch: {} - Training Cost: {}  Testing Cost: {}".format(epoch, training_cost, testing_cost))

    # Training is now complete!

    # Get the final accuracy scores by running the "cost" operation on the training and test data sets
    final_training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: y_train})
    final_testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: y_test})

    print("Final Training cost: {}".format(final_training_cost))
    print("Final Testing cost: {}".format(final_testing_cost))

    y_predicted = session.run(prediction, feed_dict={X: X_scaled_testing})
    
    inputs = {
        'input': tf.saved_model.utils.build_tensor_info(X)
        }
    outputs = {
        'earnings': tf.saved_model.utils.build_tensor_info(prediction)
        }

In [ ]:
i = 14
y_test_not_one_hot = np.argmax(y_test, axis=1)
if y_test_not_one_hot[i] == 0:
    weather = 'Clear'
elif y_test_not_one_hot[i] == 1:
    weather = 'Rain'
else:
    weather = 'Snow'

print('Sample Prediction from Neural Network:')
print('Percent Chance of Clear Weather: {}'.format(y_predicted[i][0]*100))
print('Percent Chance of Rain: {}'.format(y_predicted[i][1]*100))
print('Percent Chance of Snow: {}'.format(y_predicted[i][2]*100))
print('\n\nAcutal Weather: %s' %weather)

In [ ]:
y_predicted_not_one_hot = np.argmax(y_predicted, axis=1) 
print('Neural Network Test Set Accuracy: {}'\
      .format(np.sum((y_predicted_not_one_hot == y_test_not_one_hot) / len(y_test_not_one_hot))))

## Using Keras

In [ ]:
# define structure of model
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(n_inputs, )))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [ ]:
# compile model
adam = keras.optimizers.Adam(0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy', categorical_accuracy])

In [ ]:
model.fit(X_scaled_training, y_train, epochs=100)

In [ ]:
predictions = model.predict(X_scaled_testing)
print(model.evaluate(X_scaled_training, y_train)[0])


In [ ]:
y_predicted_not_one_hot = np.argmax(predictions, axis=1) 
y_test_not_one_hot = np.argmax(y_test, axis=1)
np.sum((y_predicted_not_one_hot == y_test_not_one_hot) / len(y_test_not_one_hot))

In [ ]:
tmp = []
for i in range(13):
    subset = df_us[df_us['num_hours']==i]
    y = to_categorical(subset['class'])
    X = subset[['temperature', 'humidity', 'mslp', 'total_cloud_cover', 'high_cloud_cover', \
                'medium_cloud_cover','low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', \
                'wind_speed_10m', 'wind_dir_10m', 'wind_speed_80m', 'wind_dir_80m', \
                'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust']].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, \
                                                        random_state=rs)
    
    X_scaler = MinMaxScaler(feature_range=(0,1))
    X_scaled_training = X_scaler.fit_transform(X_train)
    X_scaled_testing = X_scaler.transform(X_test)
    
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_inputs, )))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    
    adam = keras.optimizers.Adam(0.001)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy', categorical_accuracy])
    model.fit(X_scaled_training, y_train, epochs=100, verbose=0)
    
    train_loss = model.evaluate(X_scaled_training, y_train)[0]
    train_cat_acc = model.evaluate(X_scaled_training, y_train)[2]
    
    test_loss = model.evaluate(X_scaled_testing, y_test)[0]
    test_cat_acc = model.evaluate(X_scaled_testing, y_test)[2]
    
    tmp.append([i, train_loss, train_cat_acc, test_loss, test_cat_acc])

NN_model_performances = pd.DataFrame(tmp)
NN_model_performances.columns = ['num_hours', 'train_loss', 'train_cat_acc', 'test_loss', 'test_cat_acc']

In [ ]:
plt.figure(figsize = (10, 8))
plt.plot(NN_model_performances.num_hours, NN_model_performances.train_loss, label='Training Loss')
plt.plot(NN_model_performances.num_hours, NN_model_performances.test_loss, label='Testing Loss')
plt.plot(NN_model_performances.num_hours, NN_model_performances.train_cat_acc, label='Training Categorical Accuracy')
plt.plot(NN_model_performances.num_hours, NN_model_performances.test_cat_acc, label='Testing Categorical Accuracy')
plt.ylim([0.0, 1])
plt.grid()
plt.legend()
plt.xlabel('Number of Hours to Prediction')
plt.ylabel('Score')

plt.title('Neural Network Model Performances at Different Prediction Time Intervals');

In [ ]:
num_hidden_layer_options = [1, 2, 3]
num_nodes_options = [50, 100, 200]
max_epochs = 200
patience = 10
learning_rate = 0.001

tmp = []
for num_nodes in num_nodes_options:
    
    for i in range(len(num_hidden_layer_options)):
        model = Sequential()
        model.add(Dense(num_nodes, activation='relu', input_shape=(n_inputs, )))   
        for j in range(num_hidden_layer_options[i]-1):
            model.add(Dense(num_nodes, activation='relu'))
        
        model.add(Dense(3, activation='softmax'))
        adam = keras.optimizers.Adam(learning_rate)
        model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy', categorical_accuracy])
        early_stopping_monitor = EarlyStopping(patience=patience)
        model_hist = model.fit(X_scaled_training, y_train, epochs=max_epochs, \
                               validation_split = 0.3, callbacks=[early_stopping_monitor], verbose=0)
        
        epochs = early_stopping_monitor.stopped_epoch
        
        loss = model_hist.history['loss'][-1]
        val_loss = model_hist.history['val_loss'][-1]
        
        cat_acc = model_hist.history['categorical_accuracy'][-1]
        val_cat_acc = model_hist.history['val_categorical_accuracy'][-1]
        
        tmp.append([num_nodes, num_hidden_layer_options[i], epochs, loss, \
                    val_loss, cat_acc, val_cat_acc])
        

NN_model_tuning = pd.DataFrame(tmp)
NN_model_tuning.columns = ['num_nodes', 'num_hidden_layers', 'epochs', 'loss', \
                           'val_loss', 'cat_acc', 'val_cat_acc']

In [ ]:
NN_model_tuning

# Rain and Snow

In [ ]:
rain_df = df_us[df_us['class']==1]

X = rain_df[rain_df['num_hours']==0].drop(columns=['class', 'num_hours', 'snowfall_hr', 'rainfall_hr', \
                                                   'year', 'month', 'day', 'hour']).values
y = rain_df[rain_df['num_hours']==0]['rainfall_hr'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=rs)

X_scaler = MinMaxScaler(feature_range=(0,1))
#y_scaler = MinMaxScaler(feature_range=(0,1))

X_scaled_training = X_scaler.fit_transform(X_train)
#y_scaled_training = y_scaler.fit_transform(y_train)

X_scaled_testing = X_scaler.transform(X_test)
#y_scaled_testing = y_scaler.transform(y_test)

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_scaled_training, y_train)
lin_reg.score(X_scaled_testing, y_test)

In [ ]:
model = Sequential()
model.add(Dense(10, activation='relu', input_shape = (n_inputs,)))
model.add(Dense(10, activation='relu'))
#model.add(Dense(100, activation='relu'))
#model.add(Dense(100, activation='relu'))
model.add(Dense(1))
adam = keras.optimizers.Adam(learning_rate)
model.compile(optimizer=adam, loss='mean_squared_error')
model.fit(X_scaled_training, y_train, epochs=200, validation_split=0.3)
predictions = model.predict(X_scaled_testing)
r2_score(y_test, predictions)

In [ ]:
i=100
print(predictions[i])
print(y_test[i])

In [ ]:
df = pd.read_csv('basel_weather_7-1-09_to_7-1-19.csv', sep=';', header=10)
df = df.drop(columns=['Minute'])
column_names = ['year', 'month', 'day', 'hour', 'temperature', 'humidity', 'mslp', 'rainfall_hr', 'rainfall_lr', \
                'snowfall_hr', 'snowfall_lr', 'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
                'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
                'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust']
df.columns = column_names
df = df.drop(columns=['snowfall_lr', 'rainfall_lr'])
df['hour'] = df['hour'].map(lambda x: (x + 2) % 24)
for i, row in df.iterrows():
      if row.hour == 0 or row.hour == 1:
        df.at[i,'day'] += 1

df['class_0hr'] = df.apply(lambda x: make_classes_extra(x.rainfall_hr, x.snowfall_hr), axis = 1)
for i in range(1, 13):
    col_name = 'class_' + str(i) + 'hr'
    shiftnum = i * -1
    df[col_name] = df['class_0hr'].shift(shiftnum)
    df = df.dropna()
    df[col_name] = df[col_name].astype(int)


In [ ]:
df['class_12hr'].value_counts()


In [ ]:
df_list = []
for i in range(0, 13):
    col_name = 'class_' + str(i) + 'hr'
    cols = ['year', 'month', 'day', 'hour', 'temperature', 'humidity', 'mslp', 'rainfall_hr', \
            'snowfall_hr', 'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
            'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
            'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust', col_name]
    tmp_df = df[cols]
    tmp_df = tmp_df.rename(index=str, columns={col_name: 'class'})
    df_class_0 = tmp_df[tmp_df['class'] == 0]
    df_class_1 = tmp_df[tmp_df['class'] == 1]
    df_class_2 = tmp_df[tmp_df['class'] == 2]
    df_class_3 = tmp_df[tmp_df['class'] == 3]
    df_class_4 = tmp_df[tmp_df['class'] == 4]
    
    count_class_0, count_class_2, count_class_1, count_class_4, count_class_3 = tmp_df['class'].value_counts()
    df_under_0 = df_class_0.sample(count_class_3)
    df_under_1 = df_class_1.sample(count_class_3)
    df_under_2 = df_class_2.sample(count_class_3)
    df_under_4 = df_class_4.sample(count_class_3)
    class_df = pd.concat([df_under_0, df_under_1, df_under_2, df_class_3, df_under_4], axis=0).reset_index(drop=True)
    class_df['num_hours'] = i
    df_list.append(class_df)

df_us_extra = pd.concat(df_list).reset_index(drop=True)
df_us_extra.head()

In [ ]:
df_us_extra['class'].value_counts()

In [ ]:
# Should be: 503 * 5 * 13
len(df_us_extra)

In [ ]:
y_undersample = df_us_extra[df_us_extra['num_hours']==0]['class'].values
y_undersample_cat = to_categorical(df_us_extra[df_us_extra['num_hours']==0]['class'])
X_undersample = df_us_extra[df_us_extra['num_hours']==0][['temperature', 'humidity', 'mslp', \
                'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
                'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
                'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust']].values

X_train, X_test, y_train, y_test = train_test_split(X_undersample, y_undersample, test_size=0.3, \
                                                        random_state=rs)

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)
knn.score(X_test, y_test)

In [ ]:
y_undersample = df_us_extra[df_us_extra['num_hours']==0]['class'].values.reshape(-1,)
y_undersample_cat = to_categorical(df_us_extra[df_us_extra['num_hours']==0]['class'])
X_undersample = df_us_extra[df_us_extra['num_hours']==0][['temperature', 'humidity', 'mslp', \
                'total_cloud_cover', 'high_cloud_cover', 'medium_cloud_cover', \
                'low_cloud_cover', 'sunshine_duration', 'shortwave_radiation', 'wind_speed_10m', 'wind_dir_10m',\
                'wind_speed_80m', 'wind_dir_80m', 'wind_speed_900mb', 'wind_dir_900mb', 'wind_gust']].values

X_train, X_test, y_train, y_test = train_test_split(X_undersample, y_undersample_cat, test_size=0.3, \
                                                        random_state=rs)

X_scaler = MinMaxScaler(feature_range=(0,1))

X_scaled_training = X_scaler.fit_transform(X_train)
X_scaled_testing = X_scaler.transform(X_test)

In [ ]:
model = Sequential()
model.add(Dense(150, activation='relu', input_shape = (n_inputs,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(5, activation='softmax'))
adam = keras.optimizers.Adam(0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy', categorical_accuracy])
model.fit(X_scaled_training, y_train, epochs=100, validation_split=0.3)
predictions = model.predict(X_scaled_testing)

In [ ]:
def categorize_rainfall(rainfall_hr):
    if rainfall_hr < 0.3:
        return 0
    else:
        return 1

rain_df = rain_df.drop(columns=['class'])
rain_df['class'] = rain_df['rainfall_hr'].apply(categorize_rainfall)

In [ ]:
rain_df['class'].value_counts()

In [ ]:
X = rain_df[rain_df['num_hours']==0].drop(columns=['class', 'num_hours', 'snowfall_hr', 'rainfall_hr', \
                                                   'year', 'month', 'day', 'hour']).values
y = rain_df[rain_df['num_hours']==0]['class'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=rs)


In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)
knn.score(X_test, y_test)

In [ ]:
X = rain_df[rain_df['num_hours']==0].drop(columns=['class', 'num_hours', 'snowfall_hr', 'rainfall_hr', \
                                                   'year', 'month', 'day', 'hour']).values
y = to_categorical(rain_df[rain_df['num_hours']==0]['class'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=rs)

In [ ]:
model = Sequential()
model.add(Dense(50, activation='relu', input_shape = (n_inputs,)))
model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))
adam = keras.optimizers.Adam(0.001)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy', categorical_accuracy])
model.fit(X_train, y_train, epochs=100, validation_split=0.3)
predictions = model.predict(X_scaled_testing)